In [3]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os
from tqdm import tqdm

# Pfade zu den Dateien
satellite_image_path = "../data/timeseries/20LMP_Sat.png"
mask_folder_path = "../data/timeseries/SEN4AMA_FullProduct_150123/20LMP"
output_folder = "../data/timeseries/output_frames"

# Erstelle den Ausgabeordner, wenn er nicht existiert
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lade das ursprüngliche Satellitenbild
satellite_image = Image.open(satellite_image_path)

# Initialisiere ein leeres Bild für die kombinierte Maske
combined_mask = None

# Zähle die Anzahl der Dateien im Maskenordner
num_files = sum(1 for _ in os.listdir(mask_folder_path) if _.endswith("_CHANGE.tif"))

# Sammle die Dateien in umgekehrter Reihenfolge der Datumsangaben
file_list = sorted(os.listdir(mask_folder_path))
file_list.reverse()

# Erstelle eine Fortschrittsanzeige
with tqdm(total=num_files, desc="Processing Masks") as pbar:
    # Iteriere über die Dateien im Maskenordner
    for i, filename in enumerate(file_list):
        if filename.endswith("_CHANGE.tif"):
            # Lade die Maske und konvertiere sie zu einem NumPy-Array
            mask = Image.open(os.path.join(mask_folder_path, filename))
            mask = np.array(mask)

            # Konvertiere weiße Pixel in nicht-Null-Werte (1)
            mask[mask > 0] = 1

            # Füge die Maske zur kombinierten Maske hinzu
            if combined_mask is None:
                combined_mask = mask
            else:
                combined_mask += mask

            # Überlagere die kombinierte Maske auf das Satellitenbild
            color = np.array([0, 100, 0], dtype=np.uint8)
            color_mask = np.zeros(combined_mask.shape + (3,), dtype=np.uint8)
            color_mask[combined_mask > 0] = color

            # Konvertiere NumPy-Array in ein PIL-Bild
            color_mask_image = Image.fromarray(color_mask)

            # Passe die Größe der kombinierten Maske an die des Satellitenbildes an
            color_mask_image = color_mask_image.resize(satellite_image.size, resample=Image.NEAREST)

            # Überlagere die kombinierte Maske auf das Satellitenbild
            result = Image.blend(satellite_image.convert('RGBA'), color_mask_image.convert('RGBA'), alpha=0.3)

            # Füge das Datum hinzu
            draw = ImageDraw.Draw(result)
            font_size = 36  # Adjust the font size as needed
            date = filename.split('_')[1]
            formatted_date = f"{date[:4]}-{date[4:6]}-{date[6:]}"
            draw.text((10, result.size[1] - 40), formatted_date, fill=(255, 255, 255))

            # Speichere das Ergebnisbild im Ausgabeordner
            frame_path = os.path.join(output_folder, f"frame_{i:03d}.png")
            result.save(frame_path)

            # Aktualisiere die Fortschrittsanzeige
            pbar.update(1)

# Erstelle ein GIF aus den gespeicherten Bildern (in umgekehrter Reihenfolge)
frames = []
for i in range(num_files):
    frame_path = os.path.join(output_folder, f"frame_{i:03d}.png")
    frames.append(Image.open(frame_path))

# Speichere das GIF
gif_path = "../data/timeseries/deforestation_20LMP.gif"
frames[0].save(gif_path, save_all=True, append_images=frames[::-1], duration=50, loop=5)


Processing Masks: 100%|██████████| 140/140 [04:03<00:00,  1.74s/it]
